# 06 - Benzene

**Overview** 

This notebook guides you through the setup of quantum chemistry calculations using Python and some dedicated modules (RDKit, CirPy, Py3DMol, PySCF). The goal is to use quantum-mechanics to predict the stabilization energy of aromatic rings. For this purpose, we will simulate a benzene ring, a cyclohexane ring, and the hydrogen molecule (i.e. the hydrogenation reaction of benzene) and compare it with the hydrogenation of ethene. 

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
!pip install -q rdkit pyscf cirpy geometric py3Dmol > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import cirpy
import py3Dmol
from pyscf import mp, cc, fci, ci, gto, scf, geomopt
from pyscf.geomopt.geometric_solver import optimize
from pyscf.tools import cubegen

$$\require{mhchem}$$
**Problem** 

For this notebook, we want to simulate different molecules connected by a chemical reaction, so as to obtain information on the stability of aromatic molecules. In order to understand the stabilization effect coming from the aromatic ring, we will compare the hydrogenation reaction of benzene with the same reaction on ethene. Namely, if the three double bonds of benzene were equal to the double bond in ethene, we would expect the energy associate with 

$$\ce{3H2(g) + C6H6(g)->C6H12(g)}$$

to be three times the energy of the reaction

$$\ce{H2(g) + C2H4(g)->C2H6(g)}$$

**Model**
In order to compute the above energies we will not need to play with dangerous chemicals, but we will use quantum chemistry methods to compute the total energy of the electrons and nuclei in the different molecules involved in the reaction. We will explore using different levels of theory (HF, CI, MP2, CC) and different basis sets, to make sure we have a good understanding of the error bars associated with our predictions. 

>What level of theory and basis set will give us the best trade off between costs and accuracy? What would you consider as accurate? 

**Questions**

Before you run any simulation, answer the following question(s):

1. Given that the experimental hydrogenation energy of ethene is around 30 kcal/mol, what would you expect the hydrogenation energy of benzene to be? 
2. What amount of kcal/mol would you consider as accurate for your calculations (or for experiments)? 

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Start from the simplest and fastest method/basis set combination, neglect the effect of relaxing the atoms to their equilibrium positions, and evaluate the stabilization energy due to aromaticity in benzene. Is it qualitatively correct? 
4. Still using the fastest approach, include geometry optimization in your workflow. How much does your results change? 
5. Explore how the level of theory affects your results, while keeping a minimal basis set. How much is correlation effects relevant in your system?
6. Keep the lowest level of theory and try to converge towards the complete basis set limit. Is the basis set very relevant for this problem? Why or why not? 
7. Given the limited resources available on Colab, we do not want to perform geometry optimization with the same level of theory of the final energy calculations. It is very common to use a lower level of theory to optimize the positions of the atoms and then perform a more accurate calculation on the optimized geometry. Use this approach to push the accuracy of your calculations. 

In [ ]:
# @title Generate and Draw a Molecule via RDKit and CirPy { display-mode: "form" }
input_type = "name" # @param ["smiles", "cas", "name"]
molecule = "caffeine" # @param {type: "string"}
task = "3d" # @param ["draw", "3d", "coordinates"]

# Define molecule (e.g., ethanol)
if input_type == "smiles":
    smiles = molecule
elif input_type == "cas" or input_type == "name":
    smiles = cirpy.resolve(molecule, 'smiles')
else:
    raise ValueError("Invalid input type. Choose 'smiles', 'cas', or 'name'.")

mol = Chem.MolFromSmiles(smiles)
if task == "draw":
    # This draws the 2D structure of the molecule
    img = Draw.MolToImage(mol)
    display(img)
elif task == "coordinates":
    # This prints the atomic coordinates of the molecule
    mol = Chem.AddHs(mol)  # Add hydrogens for accurate coordinates
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    conf = mol.GetConformer()
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        print(f"{atom.GetSymbol()}: ({pos.x:.4f}, {pos.y:.4f}, {pos.z:.4f})")
elif task == "3d":
    # This generates a 3D structure of the molecule
    mol = Chem.AddHs(mol)  # Add hydrogens for accurate coordinates
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    mb = Chem.MolToMolBlock(mol)
    view = py3Dmol.view(width=400, height=300)
    view.addModel(mb, 'mol')
    view.setStyle({'stick': {}})
    view.zoomTo()
    view.show()
    print(cirpy.Molecule(molecule).twirl_url)
else:
    raise ValueError("Invalid scope. Choose 'draw', '3d', or 'coordinates'.")

In [ ]:
# @title Quantum Chemistry Calculation with PySCF { display-mode: "form" }
level = "RHF" # @param ["RHF", "UHF", "CCS", "CCSD", "CCSD(T)", "MP2", "CISD", "FCI"]
basis = "sto-3g" # @param ["sto-3g", "6-31g", "cc-pvdz", "cc-pvtz"]
# Define molecule (e.g., benzene)
input_type = "name" # @param ["smiles", "cas", "name"]
molecule = "benzene" # @param {type: "string"}
if input_type == "smiles":
    smiles = molecule
elif input_type == "cas" or input_type == "name":
    smiles = cirpy.resolve(molecule, 'smiles')
else:
    raise ValueError("Invalid input type. Choose 'smiles', 'cas', or 'name'.")

# Convert SMILES to 3D coordinates
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)  # Add hydrogens for accurate coordinates
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
conf = mol.GetConformer()
coords = []
mol_str = []
for atom in mol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    coords.append((pos.x, pos.y, pos.z))
    mol_str.append(atom.GetSymbol())

# Set up and run the quantum chemistry calculation
mol = gto.Mole()
mol.atom = list(zip(mol_str, coords))
mol.basis = basis
mol.verbose = 4
mol.build()

if level == "UHF":
    mf = scf.UHF(mol).run()
    print(f"UHF Energy: {mf.e_tot:.8f} Hartree")
else:
    mf = scf.RHF(mol).run()
    print(f"RHF Energy: {mf.e_tot:.8f} Hartree")

    if level == "MP2":
        mp2 = mp.MP2(mf).run()
        print(f"MP2 correlation energy: {mp2.e_corr:.8f} Hartree")
        print(f"MP2 total energy:       {mp2.e_tot:.8f} Hartree")

    elif level == "CCS":
        ccs = cc.CCS(mf).run()
        print(f"CCS correlation energy: {ccs.e_corr:.8f} Hartree")
        print(f"CCS total energy:       {ccs.e_tot:.8f} Hartree")

    elif level == "CCSD":
        ccsd = cc.CCSD(mf).run()
        print(f"CCSD correlation energy: {ccsd.e_corr:.8f} Hartree")
        print(f"CCSD total energy:       {ccsd.e_tot:.8f} Hartree")

    elif level == "CCSD(T)":
        ccsd = cc.CCSD(mf).run()
        ccsdt = ccsd.ccsd_t()
        total_energy = mf.e_tot + ccsd.e_corr + ccsdt
        print(f"CCSD correlation energy:  {ccsd.e_corr:.8f} Hartree")
        print(f"(T) correction:           {ccsdt:.8f} Hartree")
        print(f"CCSD(T) total energy:     {total_energy:.8f} Hartree")

    elif level == "CISD":
        cisd = ci.CISD(mf).run()
        print(f"CISD correlation energy: {cisd.e_corr:.8f} Hartree")
        print(f"CISD total energy:       {cisd.e_tot:.8f} Hartree")

    elif level == "FCI":
        fci_solver = fci.FCI(mol, mf.mo_coeff).run()
        print(f"FCI total energy:        {fci_solver.e_tot:.8f} Hartree")

    elif level != "RHF":
        raise ValueError("Invalid level of theory.")

In [ ]:
# @title Generate Cube Files { display-mode: "form" }
property = 'density' # @param ["density", "homo", "lumo"]
homo_idx = mf.mo_occ.argmax()  # OR mf.mo_occ == 2.0 → HOMO
if property == "density":
    cubegen.density(mol, 'density.cube', mf.make_rdm1())
elif property == "homo":
    cubegen.orbital(mol, 'homo.cube', mf.mo_coeff[:, homo_idx])
elif property == "lumo":
    cubegen.orbital(mol, 'lumo.cube', mf.mo_coeff[:, homo_idx + 1])
else:
    raise ValueError("Invalid property, choose between density, homo, or lumo")

In [ ]:
# @title Geometry Optimization with PySCF { display-mode: "form" }
opt_level = "DFT" # @param ["RHF", "UHF", "DFT"]
xc_functional = "B3LYP" # @param ["B3LYP", "PBE", "PBE0", "LDA", "BLYP"]
basis = "sto-3g"  # @param ["sto-3g", "6-31g", "cc-pvdz", "cc-pvtz"]
input_type = "name"  # @param ["smiles", "cas", "name"]
molecule = "benzene"  # @param {type: "string"}

from pyscf import gto, scf, dft
from pyscf.geomopt.geometric_solver import optimize
from rdkit import Chem
from rdkit.Chem import AllChem
import cirpy

# Resolve molecule
if input_type == "smiles":
    smiles = molecule
elif input_type in ("cas", "name"):
    smiles = cirpy.resolve(molecule, "smiles")
else:
    raise ValueError("Invalid input type.")

# Generate 3D coordinates from SMILES
rdmol = Chem.MolFromSmiles(smiles)
rdmol = Chem.AddHs(rdmol)
AllChem.EmbedMolecule(rdmol, AllChem.ETKDG())
conf = rdmol.GetConformer()

coords = []
mol_str = []
for atom in rdmol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    coords.append((pos.x, pos.y, pos.z))
    mol_str.append(atom.GetSymbol())

# Build PySCF molecule
mol = gto.Mole()
mol.atom = list(zip(mol_str, coords))
mol.basis = basis
mol.verbose = 4
mol.build()

# Choose SCF or DFT method
if opt_level == "RHF":
    mf = scf.RHF(mol)
elif opt_level == "UHF":
    mf = scf.UHF(mol)
elif opt_level == "DFT":
    mf = dft.RKS(mol)
    mf.xc = xc_functional.lower()  # DFT functional
else:
    raise ValueError("Unsupported method")

# Run optimization
print("Running geometry optimization...")
mol_opt = optimize(mf)

# Rerun calculation on optimized geometry
if opt_level == "RHF":
    mf_final = scf.RHF(mol_opt).run()
elif opt_level == "UHF":
    mf_final = scf.UHF(mol_opt).run()
elif opt_level == "DFT":
    mf_final = dft.RKS(mol_opt)
    mf_final.xc = xc_functional.lower()
    mf_final = mf_final.run()

# Output optimized results
print(f"\nOptimized energy: {mf_final.e_tot:.8f} Hartree")
print("Optimized geometry (Angstrom):")
for sym, xyz in zip([a[0] for a in mol_opt.atom], mol_opt.atom_coords()):
    print(f"{sym:2}  {xyz[0]:10.5f}  {xyz[1]:10.5f}  {xyz[2]:10.5f}")

## Part 2: Semiempirical Methods (Huckel)

The Huckel model is a semi-empirical quantum mechanical model that has been developed to describe conjugted systems. While the model relies on some adjustable parameters and involves several possibly strong assumptions, it allows to reproduce some key properties of conjugated and aromatic molecules.

You can learn more on the theory and assumptions of the model in the lectures and in the following online resources:
* MIT Physical Chemistry on [The Huckel Molecular Orbital Theory](https://dspace.mit.edu/bitstream/handle/1721.1/120336/5-61-fall-2013/contents/lecture-notes/MIT5_61F13_Lecture27-28.pdf)
* P-Chem Lab from Duke University [The Huckel Approximation](https://chem.libretexts.org/Courses/Duke_University/CHEM310L_-_Physical_Chemistry_I_Lab_Manual/04%3A_Absorption_Spectrum_of_Conjugated_Dyes/4.07%3A_Appendix_B_-_The_Huckel_Approximation)
* Columbia Notes on [The Huckel Approximation](http://www.columbia.edu/itc/chemistry/chem-c2407_archive/recitations/huckel.pdf)


The key component of the Huckel model is the fact that the molecular orbitals are univoquely determined by the topology of the conjugated network. The model assumes that we only consider the $p_z$ orbitals of each conjugated atom and we build molecular orbitals from them. As a further approximation, only atomic orbitals that are in connected atoms are allowed to 'interact'. Eventually, we assume that all atoms and bonds are equivalent, so that the relative components of the molecular Hamiltonian are identical. The Hamiltonian of the system can thus be represented as a matrix, where diagonal cells $H_{ii}$ have identical values, while off-diagonal elements $H_{ij}$ are different from zero only if there is a bond between atom $i$ and atom $j$. For carbon based molecules, the model only relies on two parameters $\alpha$ and $\beta$. However, the difference in energy between the electronic states only depends on the latter parameter, which is usually estimated to be $\beta\approx-70.4\; kcal/mol$.

Molecular orbitals are given by the linear combinations of atomic orbitals that make the Hamiltonian diagonal and minimize the energy of the system. Given the regular structure of our hamiltonian, which for linear molecules is tridiagonal, we can actually solve for the eigenvalues analytically, obtaining a formula

$$ E_j=\alpha+2\beta\cos\left(\frac{\pi}{N+1}J\right)$$

with $J=1,2,\dots ,N$. However, the Huckel model can be automatically extended to account for the aromatic rings and it can, with additional parameters, be extended to include heteroatoms. 

**Questions**

Before you run any simulation, answer the following question(s):

1. Which of the two Huckel parameters ($\alpha$ and $\beta$) do you expect to affect the aromatic stabilization energy the most? 

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

2. Assuming that the sum of the $p_z$ orbital energies is the energy involved in breaking those bonds (i.e. they are the key component of the hydrogenation energy), can the Huckel model predict the aromatic stabilization? 
3. What value should the Huckel parameters have in order to reproduce the accurate quantum chemical results obtained in the previous section? 

In [ ]:
# @title Semiempirical Huckel Calculations { display-mode: "form" }
n_conjugated = 4 # @param {type: "integer"}
ring = False # @param {type: "boolean"}
alpha = -2 # @param {type: "number"}
beta = -1 # @param {type: "number"}
if ring:
    if n_conjugated < 3:
        raise ValueError("A ring requires at least 3 atoms.")

# Construct Hückel Hamiltonian
H = np.zeros((n_conjugated, n_conjugated))
np.fill_diagonal(H, alpha)
for i in range(n_conjugated - 1):
    H[i, i + 1] = beta
    H[i + 1, i] = beta
if ring:
    H[0, -1] = beta
    H[-1, 0] = beta

# Diagonalize
e_vals, e_vecs = np.linalg.eigh(H)
idx = np.argsort(e_vals)
e_vals = e_vals[idx]
e_vecs = e_vecs[:, idx]

# Layout configuration
n_mos = n_conjugated
n_cols = min(n_mos, 6)
n_rows = int(np.ceil(n_mos / n_cols))

fig = plt.figure(figsize=(3*n_cols, 6 + 2.5*n_rows))
gs = gridspec.GridSpec(n_rows + 1, n_cols, height_ratios=[2] + [1]*n_rows)

# Plot Hamiltonian heatmap
from matplotlib import cm
ax0 = fig.add_subplot(gs[0, 0])
vmin, vmax = H.min(), 0  # scale from most negative to 0
im = ax0.imshow(H, cmap=cm.Greens_r, vmin=vmin, vmax=vmax)
ax0.set_title("Hückel Hamiltonian")
ax0.set_xticks(range(n_conjugated))
ax0.set_yticks(range(n_conjugated))
fig.colorbar(im, ax=ax0, fraction=0.046, pad=0.04, label="Energy")

# Plot energy levels (right, top row)
ax1 = fig.add_subplot(gs[0, 1:])
for i, e in enumerate(e_vals):
    ax1.hlines(e, i - 0.3, i + 0.3, color='black')
ax1.set_title("MO Energy Levels")
ax1.set_ylabel("Energy")
ax1.set_xticks([])
ax1.grid(axis='y', linestyle='--', alpha=0.5)

# Plot MOs (bottom grid)
for i in range(n_mos):
    row = i // n_cols + 1
    col = i % n_cols
    ax = fig.add_subplot(gs[row, col])
    coeffs = e_vecs[:, i]
    colors = ['blue' if c >= 0 else 'red' for c in coeffs]
    ax.bar(range(n_conjugated), coeffs, color=colors)
    ax.set_title(f"MO {i+1}\nE={e_vals[i]:.2f}", fontsize=10)
    ax.set_xticks(range(n_conjugated))
    ax.set_ylim(-1.1, 1.1)
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=6)

fig.suptitle("Hückel π-System Analysis", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

n_electrons = n_conjugated  # You can change this manually if needed
if n_electrons % 2 != 0:
    raise ValueError("This code assumes an even number of electrons (closed shell).")

n_occupied = n_electrons // 2
total_energy = 2 * np.sum(e_vals[:n_occupied])

print(f"Total π-electron energy for {n_electrons} electrons: {total_energy:.4f} energy units")
